In [1]:
from api_endpoints import api_teams

In [2]:
teams = api_teams()
teams_id = []
for team in teams["teams"]:
    teams_id.append(team["id"])

KeyError: 'teams'

In [3]:
teams

{'message': 'Limit Exceeded'}

In [3]:
print(len(teams_id), len(teams_id[0]))

535 36


In [4]:
from api_endpoints import api_team_roasters

First, fill the parent tables of team
1. from team roasters ep  
    1. conferences
    2. divisions
    3. venues
2. then teams

In [9]:
roaster_info = api_team_roasters(teams_id[0])


In [10]:
roaster_info

{'id': '30abfee7-57a7-4984-9a1c-55cb28970da3',
 'name': '49ers',
 'market': 'Charlotte',
 'alias': 'CHAR',
 'founded': 1946,
 'mascot': 'Norm the Niner',
 'fight_song': 'Charlotte 49ers Fight Song',
 'championships_won': 0,
 'conference_titles': 0,
 'playoff_appearances': 0,
 'franchise': {'id': '30abfee7-57a7-4984-9a1c-55cb28970da3', 'name': '49ers'},
 'venue': {'id': '5189127d-ea58-4917-84b8-8456f3abb8b2',
  'name': 'Jerry Richardson Stadium',
  'city': 'Charlotte',
  'state': 'NC',
  'country': 'USA',
  'zip': '28262',
  'address': '9201 University City Boulevard',
  'capacity': 15314,
  'surface': 'artificial',
  'roof_type': 'outdoor',
  'location': {'lat': '35.3105', 'lng': '-80.7401'}},
 'division': {'id': '43d3cede-6b8f-4158-bc7f-c3c7365e5b32',
  'name': 'I-A',
  'alias': 'FBS'},
 'conference': {'id': '49602742-5d66-412e-8a58-ab1e613d593c',
  'name': 'American',
  'alias': 'AAC'},
 'coaches': [{'id': '35ccf611-95e3-40b0-8de1-986a72aa5020',
   'full_name': 'Tim Albin',
   'first

In [11]:
print(type(roaster_info["founded"]))
print(type(roaster_info["championships_won"]))
print(type(roaster_info["venue"]['zip']))

<class 'int'>
<class 'int'>
<class 'str'>


In [12]:
import pandas as pd

In [ ]:
venue_df = pd.DataFrame(columns=["venue_id", "name","city", "state", "country", "zip", "address", "capacity", "surface", "roof_type", "latitude", "longitude"])
conferene_df = pd.DataFrame(columns=["conference_id", "name", "alias"])
divison_df = pd.DataFrame(columns=["division_id", "name", "alias"])
teams_df = pd.DataFrame(columns= ['team_id', 'market', 'name', 'alias', 'founded', 'mascot', 'fight_song', 'championships_won', 'conference_id', 'division_id', 'venue_id'])
players_df = pd.DataFrame(columns=['player_id', 'first_name', 'last_name', 'abbr_name', 'birth_place', 'position', 'height', 'weight', 'status', 'eligibility', 'team_id'])
coaches_df = pd.DataFrame(columns=["coach_id", "full_name", "position", "team_id"])

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os, time, random

def _fetch_roaster(tid, max_retries=6, base_backoff=1.0):
    backoff = base_backoff
    for attempt in range(1, max_retries + 1):
        try:
            res = api_team_roasters(tid)
        except Exception as e:
            msg = str(e)
            # treat typical rate-limit errors as retryable
            if "Too Many Requests" in msg or "429" in msg:
                wait = backoff + random.uniform(0, 0.5)
                time.sleep(wait)
                backoff *= 2
                continue
            return {"id": tid, "_error": msg}

        # api wrapper may return a dict with a 'message' or '_error'
        if isinstance(res, dict) and (res.get("message") == "Too Many Requests" or
                                      "Too Many Requests" in str(res.get("_error", ""))):
            # if the API returned Retry-After in headers, use that (needs wrapper change to expose headers)
            wait = backoff + random.uniform(0, 0.5)
            time.sleep(wait)
            backoff *= 2
            continue

        return res

    return {"id": tid, "_error": "TooManyRequests_after_retries"}

# lower concurrency to avoid bursts
max_workers = min(10, (os.cpu_count() or 1) * 2, len(teams_id))
roasters_list = []
with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = [ex.submit(_fetch_roaster, tid) for tid in teams_id]
    for i, fut in enumerate(as_completed(futures), 1):
        roasters_list.append(fut.result())
        if i % 50 == 0 or i == len(teams_id):
            print(f"Fetched {i}/{len(teams_id)}")

Fetched 50/535
Fetched 100/535
Fetched 150/535
Fetched 200/535
Fetched 250/535
Fetched 300/535
Fetched 350/535
Fetched 400/535
Fetched 450/535
Fetched 500/535
Fetched 535/535


In [14]:
roasters_list[534]

{'id': 'c8a2e426-e152-4c79-8f7c-33372a42e1a7',
 '_error': 'TooManyRequests_after_retries'}

In [ ]:
def _get(d, keys, default=None):
    """Safe nested dict getter: _get(d, ['a','b']) -> d['a']['b'] or default."""
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

for roaster in roasters_list:
    # skip non-roaster results (e.g. error messages)
    if not isinstance(roaster, dict) or 'id' not in roaster:
        continue

    venue_dict = {
        'venue_id': _get(roaster, ['venue', 'id']),
        'name': _get(roaster, ['venue', 'name']),
        'city': _get(roaster, ['venue', 'city']),
        'state': _get(roaster, ['venue', 'state']),
        'country': _get(roaster, ['venue', 'country']),
        'zip': _get(roaster, ['venue', 'zip']),
        'address': _get(roaster, ['venue', 'address']),
        'capacity': _get(roaster, ['venue', 'capacity']),
        'surface': _get(roaster, ['venue', 'surface']),
        'roof_type': _get(roaster, ['venue', 'roof_type']),
        'latitude': _get(roaster, ['venue', 'location', 'lat']),
        'longitude': _get(roaster, ['venue', 'location', 'lng']),
    }

    conference_dict = {
        'conference_id': _get(roaster, ['conference', 'id']),
        'name': _get(roaster, ['conference', 'name']),
        'alias': _get(roaster, ['conference', 'alias']),
    }

    division_dict = {
        'division_id': _get(roaster, ['division', 'id']),
        'name': _get(roaster, ['division', 'name']),
        'alias': _get(roaster, ['division', 'alias']),
    }

    teams_dict = {
        'team_id': _get(roaster, ['id']),
        'market': _get(roaster, ['market']),
        'name': _get(roaster, ['name']),
        'alias': _get(roaster, ['alias']),
        'founded': _get(roaster, ['founded']),
        'mascot': _get(roaster, ['mascot']),
        'fight_song': _get(roaster, ['fight_song']),
        'championships_won': _get(roaster, ['championships_won']),
        'conference_id': _get(roaster, ['conference', 'id']),
        'division_id': _get(roaster, ['division', 'id']),
        'venue_id': _get(roaster, ['venue', 'id']),
    }

    # Insert only when id is present and not already in target DF
    if venue_dict['venue_id'] is not None and venue_dict['venue_id'] not in venue_df['venue_id'].values:
        venue_df.loc[len(venue_df)] = venue_dict

    if conference_dict['conference_id'] is not None and conference_dict['conference_id'] not in conferene_df['conference_id'].values:
        conferene_df.loc[len(conferene_df)] = conference_dict

    if division_dict['division_id'] is not None and division_dict['division_id'] not in divison_df['division_id'].values:
        divison_df.loc[len(divison_df)] = division_dict

    if teams_dict['team_id'] is not None and teams_dict['team_id'] not in teams_df['team_id'].values:
        teams_df.loc[len(teams_df)] = teams_dict

    for coach in roaster.get('coaches', []) or []:
        c = {
            'coach_id': _get(coach, ['id']),
            'full_name': _get(coach, ['full_name']),
            'position': _get(coach, ['position']),
            'team_id': teams_dict['team_id'],
        }
        if c['coach_id'] is not None and c['coach_id'] not in coaches_df['coach_id'].values:
            coaches_df.loc[len(coaches_df)] = c

    for player in roaster.get('players', []) or []:
        p = {
            'player_id': _get(player, ['id']),
            'first_name': _get(player, ['first_name']),
            'last_name': _get(player, ['last_name']),
            'abbr_name': _get(player, ['abbr_name']),
            'birth_place': _get(player, ['birth_place']),
            'position': _get(player, ['position']),
            'height': _get(player, ['height']),
            'weight': _get(player, ['weight']),
            'status': _get(player, ['status']),
            'eligibility': _get(player, ['eligibility']),
            'team_id': teams_dict['team_id'],
        }
        if p['player_id'] is not None and p['player_id'] not in players_df['player_id'].values:
            players_df.loc[len(players_df)] = p


/tmp/ipykernel_32872/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  players_df.loc[len(players_df)] = p
/tmp/ipykernel_32872/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  players_df.loc[len(players_df)] = p
/tmp/ipykernel_32872/1783081211.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain t

In [13]:
from sqlalchemy import create_engine

In [ ]:
from sql_connection import main

In [15]:
main()

Creating ncaafb_db
Loading schema file from ./schema.sql
Database ncaafb_db is hot and fresh, ready to kill
Accessing ncaafb_db database...


In [16]:
engine = create_engine(f"mysql+mysqlconnector://{USER_NAME}:{USER_PASSWORD}@{HOST_ID}/{DB_NAME}")

## create table for l1 parents without drop or replacing it or otherwise sql not allow to drop linked tables.
- level1 parent includes: 1. conference (done) 2. venue (done) 3. division (done)
- level2 parent includes: 1. teams (done) 2. seasons (DONE)
- level3 parent includes: 1. rankings 2. player stats () 3. player (done) 4. coaches (done)


In [17]:
conferene_df.to_sql("CONFERENCES", engine, index=False, if_exists="append")
venue_df.to_sql("VENUES", engine, index=False, if_exists="append")
divison_df.to_sql("DIVISIONS", engine, index=False, if_exists="append")
teams_df.to_sql("TEAMS", engine, index=False, if_exists="append")
players_df.to_sql("PLAYERS", engine, index=False, if_exists="append")
coaches_df.to_sql("COACHES", engine, index=False, if_exists="append")

-1

In [18]:
# from sqlalchemy import text
# with engine.connect() as conn:
#     conn.execute(text("SET FOREIGN_KEY_CHECKS=0"))
#     # conferene_df.to_sql("CONFERENCES", conn, index=False, if_exists="replace")
#     conn.execute(text("drop table if exists CONFERENCES"))
#     conn.execute(text("SET FOREIGN_KEY_CHECKS=1"))

In [19]:
# SELECT
# TEAMS.team_id,
# CONFERENCES.name,
# DIVISIONS.name
# FROM TEAMS 
# JOIN CONFERENCES ON TEAMS.conference_id = TEAMS.conference_id
# JOIN DIVISIONS ON TEAMS.division_id = DIVISIONS.division_id


In [25]:
from api_endpoints import api_season_details

In [26]:
season_details = api_season_details()

In [27]:
season_details

{'league': {'id': '26c1246a-2fc3-4b7e-8999-1685d3ab4676',
  'name': 'NCAA Football',
  'alias': 'NCAAFB'},
 'seasons': [{'id': '7010df8f-1e08-4021-91b8-ed09676c33de',
   'year': 2013,
   'start_date': '2013-08-29',
   'end_date': '2014-01-14',
   'status': 'closed',
   'type': {'code': 'REG'}},
  {'id': 'b07de423-006d-4ae2-b9ed-532841ab1c4c',
   'year': 2014,
   'start_date': '2014-08-23',
   'end_date': '2015-01-19',
   'status': 'closed',
   'type': {'code': 'REG'}},
  {'id': '23e2cfa5-e19f-4ec1-bc60-b568212d91e5',
   'year': 2015,
   'start_date': '2015-08-29',
   'end_date': '2016-01-25',
   'status': 'closed',
   'type': {'code': 'REG'}},
  {'id': '9ff91613-d2c8-4922-9f51-2ce706f5ac0b',
   'year': 2016,
   'start_date': '2016-08-27',
   'end_date': '2017-01-16',
   'status': 'closed',
   'type': {'code': 'REG'}},
  {'id': '8e84b9ee-9b8e-45ef-b61f-0dfd26ff37e6',
   'year': 2017,
   'start_date': '2017-08-26',
   'end_date': '2018-01-15',
   'status': 'closed',
   'type': {'code': '

In [28]:
season_list = season_details['seasons']
type(season_list)

list

In [29]:
seasons_df = pd.DataFrame(columns=["season_id", "year", "start_date", "end_date", "status", "type_code"])

In [ ]:


for season in season_list:
    season_dict = {
        "season_id": _get(season, ["id"]),
        "year": _get(season, ["year"]),
        "start_date": _get(season, ["start_date"]),
        "end_date": _get(season, ["end_date"]),
        "status": _get(season, ["status"]),
        "type_code": _get(season, ["type", "code"]),
    }

    if season_dict["season_id"] is not None and season_dict["season_id"] not in seasons_df["season_id"].values:
        seasons_df.loc[len(seasons_df)] = season_dict


In [31]:
seasons_df

,season_id,year,start_date,end_date,status,type_code
0,7010df8f-1e08-4021-91b8-ed09676c33de,2013,2013-08-29,2014-01-14,closed,REG
1,b07de423-006d-4ae2-b9ed-532841ab1c4c,2014,2014-08-23,2015-01-19,closed,REG
2,23e2cfa5-e19f-4ec1-bc60-b568212d91e5,2015,2015-08-29,2016-01-25,closed,REG
3,9ff91613-d2c8-4922-9f51-2ce706f5ac0b,2016,2016-08-27,2017-01-16,closed,REG
4,8e84b9ee-9b8e-45ef-b61f-0dfd26ff37e6,2017,2017-08-26,2018-01-15,closed,REG
5,34e9387d-2422-40dd-b289-572245995f35,2018,2018-08-25,2019-01-14,closed,REG
6,a89f105a-fe49-4331-b638-4ee12dae0493,2019,2019-08-24,2020-01-20,closed,REG
7,17d91be7-4bd3-4407-80e2-eca205c9284a,2020,2020-08-29,2021-05-24,closed,REG
8,d1132b13-2c29-4e6d-9011-b627c7fd9a17,2021,2021-08-28,2022-01-24,closed,REG
9,ff53e5e4-10ef-4842-a0b7-b489956fa07e,2022,2022-08-27,2023-01-16,closed,REG


In [32]:
seasons_df.to_sql("SEASONS", engine, index=False, if_exists="append")

-1

In [68]:
players_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2538 entries, 0 to 2537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_id    2538 non-null   object 
 1   first_name   2538 non-null   object 
 2   last_name    2538 non-null   object 
 3   abbr_name    2538 non-null   object 
 4   birth_place  2283 non-null   object 
 5   position     2538 non-null   object 
 6   height       2317 non-null   object 
 7   weight       2310 non-null   float64
 8   status       2538 non-null   object 
 9   eligibility  2369 non-null   object 
 10  team_id      2538 non-null   object 
dtypes: float64(1), object(10)
memory usage: 237.9+ KB


In [84]:
players_id = players_df["player_id"].values

In [85]:
players_id[0]

'0df3ae40-74bd-11f0-addf-d95e01ada62f'

In [86]:
from api_endpoints import api_player_profile
import time

def _fetch_player_profile(pid, max_retries=5):
    for attempt in range(max_retries):
        try:
            return api_player_profile(pid)
        except Exception as e:
            msg = str(e)
            # backoff on rate limit-ish errors
            if "Too Many Requests" in msg or "429" in msg:
                # time.sleep((2 ** attempt) * 0.1 + 0.1)
                time.sleep(2)
                continue
            return {"player_id": pid, "_error": msg}
    return {"player_id": pid, "_error": "TooManyRequests_after_retries"}

max_workers = min(50, (os.cpu_count() or 1) * 5, len(players_id))
playerprof_list = []
with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = {ex.submit(_fetch_player_profile, pid): pid for pid in players_id}
    for i, fut in enumerate(as_completed(futures), 1):
        playerprof_list.append(fut.result())
        if i % 100 == 0 or i == len(players_id):
            print(f"Fetched {i}/{len(players_id)}")

Fetched 100/2538
Fetched 200/2538
Fetched 300/2538
Fetched 400/2538
Fetched 500/2538
Fetched 600/2538
Fetched 700/2538
Fetched 800/2538
Fetched 900/2538
Fetched 1000/2538
Fetched 1100/2538
Fetched 1200/2538
Fetched 1300/2538
Fetched 1400/2538
Fetched 1500/2538
Fetched 1600/2538
Fetched 1700/2538
Fetched 1800/2538
Fetched 1900/2538
Fetched 2000/2538
Fetched 2100/2538
Fetched 2200/2538
Fetched 2300/2538
Fetched 2400/2538
Fetched 2500/2538
Fetched 2538/2538


In [92]:
c = 0
for p in playerprof_list:
    if _get(p, ["message"]) == "Too Many Requests":
        c += 1
print(c)

2411


In [63]:
playerstat_df = pd.DataFrame(columns=['player_id', 'team_id', 'season_id', 'games_played', 'games_started', 'rushing_yards', 'rushing_touchdowns', 'receiving_yards', 'receiving_touchdowns', 'kick_return_yards', 'fumbles'])

In [ ]:
for player in playerprof_list:
    playerstat_dict = {key:None for key in playerstat_df.columns}
    playerstat_dict["player_id"] = _get(player, ["id"])
    playerstat_dict["team_id"] = _get(player, ["team", "id"])
    if _get(player, ["seasons"]) in [None, []]:
        playerstat_df.loc[len(playerstat_df)] = playerstat_dict
        continue

    for season in player["seasons"]:
        playerstat_dict["season_id"] = _get(season, ["id"])
        if _get(season, ["teams"]) in [None, []]:
            playerstat_df.loc[len(playerstat_df)] = playerstat_dict
            continue

        for team in season["teams"]:
            if playerstat_dict["team_id"] is not None and _get(team, ["id"]) == playerstat_dict["team_id"]:
                playerstat_dict["team_id"] = _get(team, ["id"])
                playerstat_dict["games_played"] = _get(team, ["statistics", "games_played"])
                playerstat_dict["games_started"] = _get(team, ["statistics", "games_started"])
                playerstat_dict["rushing_yards"] = _get(team, ["statistics", "rushing", "yards"])
                playerstat_dict["rushing_touchdowns"] = _get(team, ["statistics", "rushing", "touchdowns"])
                playerstat_dict["receiving_yards"] = _get(team, ["statistics", "receiving", "yards"])
                playerstat_dict["receiving_touchdowns"] = _get(team, ["statistics", "receiving", "touchdowns"])
                playerstat_dict["kick_return_yards"] = _get(team, ["statistics", "kick_returns", "yards"])
                playerstat_dict["fumbles"] = _get(team, ["statistics", "fumbles", "fumbles"])

                playerstat_df.loc[len(playerstat_df)] = playerstat_dict




In [65]:
playerstat_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2599 entries, 0 to 2598
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   player_id             159 non-null    object
 1   team_id               159 non-null    object
 2   season_id             159 non-null    object
 3   games_played          159 non-null    object
 4   games_started         159 non-null    object
 5   rushing_yards         17 non-null     object
 6   rushing_touchdowns    17 non-null     object
 7   receiving_yards       34 non-null     object
 8   receiving_touchdowns  34 non-null     object
 9   kick_return_yards     12 non-null     object
 10  fumbles               21 non-null     object
dtypes: object(11)
memory usage: 243.7+ KB


In [66]:
playerstat_df

,player_id,team_id,season_id,games_played,games_started,rushing_yards,rushing_touchdowns,receiving_yards,receiving_touchdowns,kick_return_yards,fumbles
0,3ece6d70-0fd2-11f0-afee-8b37bdbeabef,30abfee7-57a7-4984-9a1c-55cb28970da3,754e4990-efc7-11ef-bb2a-5d2d22b9215e,4,0,None,None,30,1,None,None
1,2c0ef5c8-26e3-495a-83c1-c5dfaf726332,30abfee7-57a7-4984-9a1c-55cb28970da3,908fbc20-f5c7-11ee-a306-c311afc28263,3,0,None,None,None,None,None,None
2,2c0ef5c8-26e3-495a-83c1-c5dfaf726332,30abfee7-57a7-4984-9a1c-55cb28970da3,754e4990-efc7-11ef-bb2a-5d2d22b9215e,9,0,None,None,None,None,None,None
3,109eeb62-2c58-4600-a9e1-22cd82316dde,30abfee7-57a7-4984-9a1c-55cb28970da3,d1132b13-2c29-4e6d-9011-b627c7fd9a17,11,6,None,None,None,None,None,None
4,109eeb62-2c58-4600-a9e1-22cd82316dde,30abfee7-57a7-4984-9a1c-55cb28970da3,ff53e5e4-10ef-4842-a0b7-b489956fa07e,11,7,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2594,None,None,None,None,None,None,None,None,None,None,None
2595,None,None,None,None,None,None,None,None,None,None,None
2596,None,None,None,None,None,None,None,None,None,None,None
2597,None,None,None,None,None,None,None,None,None,None,None


In [62]:
# Get all valid team_ids from the TEAMS table
valid_team_ids = set(pd.read_sql("SELECT team_id FROM TEAMS", engine)["team_id"])

# Filter playerstat_df to only include rows with valid team_id
playerstat_df = playerstat_df[playerstat_df["team_id"].isin(valid_team_ids)]

# Optionally, drop rows with missing team_id
playerstat_df = playerstat_df.dropna(subset=["team_id"])

# Now insert
playerstat_df.to_sql("PLAYER_STATISTICS", engine, index=False, if_exists="append")

-1